In [88]:
#imports
import pandas as pd
import csv
from csv import writer
from io import BytesIO

In [46]:
#load and reshape data
housing = pd.read_csv('./dataSets/housing/density_2010.csv', dtype={'GCT_STUB.target-geo-id2' : str })
income = pd.read_csv('./dataSets/full_income.csv', dtype={'FIPS' : str}).set_index(['FIPS'])
violations = pd.read_csv('./dataSets/all_violations.csv', low_memory=False)
violations['year'] = violations['Compliance Period Begin Date'].apply(lambda v : v[-4:])
state_abbrevs = pd.read_csv('./dataSets/state_abbrevs.csv', index_col=False)
counties = pd.read_csv('./dataSets/county_table.csv', dtype={'State FIPS Code' : str, 'County FIPS Code' : str})
pws_counties = pd.read_csv('./dataSets/geocoded_pws_ids_w_county.csv', dtype={'COUNTY': str})
water_systems = pd.read_csv('./dataSets/water_systems.csv')
geocoded_pws = pd.read_csv('dataSets/all_geocode_pws_fips.csv', dtype={ 'FIPS' : str })
#get just the postal code for each state
state_fips = counties[['State Postal Code', 'State FIPS Code']].drop_duplicates(subset=['State Postal Code'])

In [74]:
#preparing data for look up

#violations
#group by year
vio_year = violations.drop_duplicates()
vio_year = vio_year[['PWS ID', 'year']]
vio_year = vio_year.groupby(['PWS ID', 'year']).size().reset_index(name='counts')
vio_year = vio_year.set_index(['PWS ID', 'year'])

#housing
#make column names friendler
housing['FIPS'] = housing['GCT_STUB.target-geo-id2']
housing['housing_density'] = housing['SUBHD0402']
housing = housing.set_index('FIPS')

#water systems
#get their FIPS
water_systems = water_systems.sort_values(by='PWS ID')
water_geo = pd.merge(water_systems, geocoded_pws[['PWS ID', 'FIPS']], on='PWS ID')


In [67]:
# #merge Violations
# result = pd.merge(violations, pws_counties, on='PWS ID')
# result = pd.merge(result, state_abbrevs, on='Primacy Agency')
# result = pd.merge(result, state_fips, on='State Postal Code')
# result['FIPS'] = result['State FIPS Code'] + result['COUNTY']
# result['year'] = result['Violation First Reported Date'].apply(lambda x : x[-4:])
# result['median_income_2015'] = 0.0
# len(result.index)

# vio_year.head()
samples = water_geo.head(2)
for pws in samples.itertuples():
    print(pws)


Pandas(Index=0, _1='AL0000002', _2='AUTAUGAVILLE WATER SYSTEM', _3='Alabama', _4='Local government', _5='Ground water', _6='2,088', _7='-', _8='P O BOX 237', _9='AUTURGAVILLE', _10='AL', _11='36003', _12='05-MAR-1980', _13='11-MAY-2018', FIPS='01001')
Pandas(Index=1, _1='AL0000003', _2='BILLINGSLEY WATER SYSTEM', _3='Alabama', _4='Local government', _5='Ground water', _6='1,473', _7='Town Phone/Fax # 205-755-6031 PO Box 142', _8='P. O. BOX 143', _9='BILLINGSLEY', _10='AL', _11='36006', _12='05-MAR-1980', _13='11-MAY-2018', FIPS='01001')


In [94]:
# goal data set:
# PWS_ID, year, total_violations, median_income, housing_density
# df = pd.DataFrame(columns=['PWS ID', 'year', 'total_violations', 'median_income', 'housing_density'])

# csv_writer = writer(output)

years = list(map(lambda y : str(y), range(1982, 2016)))

csv_writer = None
with open('full_water_quality_dataset.csv', 'w') as csvfile:
    csv_writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    for pws in water_geo.itertuples():
        _id = getattr(pws, '_1')
        _fips = getattr(pws, 'FIPS')
        for i, _year in enumerate(years):
            _total_violations = 0
            _median_income = float('nan')
            _housing_density = float('nan')
            if (_id, _year) in vio_year.index:
                _total_violations = vio_year.loc[(_id, _year)]['counts']
            if (_fips) in income.index:
                _median_income = income.loc[(_fips)][_year]
            if (_fips) in housing.index:
                _housing_density = housing.loc[(_fips)]['housing_density']  
            row = [_id,_year,_total_violations,_median_income,_housing_density]
            csv_writer.writerow(row)


In [73]:
housing.head()

,GEO.id,GEO.id2,GEO.display-label,GCT_STUB.target-geo-id,GCT_STUB.target-geo-id2,GCT_STUB.display-label,GCT_STUB.display-label.1,HD01,HD02,SUBHD0301,SUBHD0302,SUBHD0303,SUBHD0401,SUBHD0402,FIPS,housing_density
0,0100000US,NaN,United States,0100000US,NaN,United States,United States,308745538,131704730,3796742.23,264836.79,3531905.43,87.4,37.3,NaN,37.3
1,0100000US,NaN,United States,0400000US01,01,United States - Alabama,Alabama,4779736,2171853,52420.07,1774.74,50645.33,94.4,42.9,01,42.9
2,0100000US,NaN,United States,0500000US01001,01001,United States - Alabama - Autauga County,Autauga County,54571,22135,604.39,9.95,594.44,91.8,37.2,01001,37.2
3,0100000US,NaN,United States,0500000US01003,01003,United States - Alabama - Baldwin County,Baldwin County,182265,104061,2027.31,437.53,1589.78,114.6,65.5,01003,65.5
4,0100000US,NaN,United States,0500000US01005,01005,United States - Alabama - Barbour County,Barbour County,27457,11829,904.52,19.64,884.88,31.0,13.4,01005,13.4
